In [1]:
%pylab inline
%matplotlib inline

import os
import time
import collections
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import emoji

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")

import paths
import emolex
import tokenizer
import useful_methods

Populating the interactive namespace from numpy and matplotlib


## All Game Emojis

In [ ]:
# read Game Infos
dfGameInfo = useful_methods.ReadGameInfoDfDropNan()

In [ ]:
# extract emojis as list
def EmojisSingleGame(week, home_team, away_team):
    # single game
    df = useful_methods.SingleGameDf(week, home_team, away_team, filtering=True, retweet=False)

    # add emoji column
    df['emoji'] = [tokenizer.EmojiText(text) for text in df.text]

    # extract emoji
    emoji_list = list(df[df['emoji'] != ""]['emoji'])

    all_emoji = []
    for emoji in emoji_list:
        emojis = emoji.split(',')
        all_emoji += emojis

    return all_emoji

In [ ]:
all_game_emoji = []

for ith_row in range(len(dfGameInfo)):
    week = dfGameInfo.loc[ith_row]['GW']
    home_team = dfGameInfo.loc[ith_row]['home_team']
    away_team = dfGameInfo.loc[ith_row]['away_team']
    
    # emoji list
    emoji_list = EmojisSingleGame(week, home_team, away_team)
    
    all_game_emoji += emoji_list

In [ ]:
# Emoji Info
print("All Emoji: ", len(all_game_emoji))
print("All Unique emoji: ", len(set(all_game_emoji)))

In [ ]:
counted = collections.Counter(all_game_emoji)
counted.most_common(10)

## Count Emoji Single Game

In [8]:
week = 13
home_team = 'City'
away_team = 'Liverpool'
FILTER_STATUS = True
RETWEET_STATUS = False
START_TIME = 1
END_TIME = 63
TIME_LIMIT = 63

In [3]:
# read single game
df = useful_methods.SingleGameDf(week, home_team, away_team, filtering=FILTER_STATUS, retweet=RETWEET_STATUS)
df['ith_minute'] = [int(item) for item in df['ith_minute']]

In [10]:
# Count Home, Away Emolex
def CountGameEmolexEmoji(week, team_home, team_away):

    # Read Single as DF
    dfGame = useful_methods.SingleGameDf(week, team_home, team_away, filtering=FILTER_STATUS, retweet=RETWEET_STATUS)
    if dfGame is None:
        return (None, None)

    dfGame.ith_minute = [int(ith_minute) for ith_minute in list(dfGame.ith_minute)]


    # Count Emoji Sentiment Count
    dic_emoji_sent_home = useful_methods.EmojiSentimentCount(dfGame, 'home', START_TIME, END_TIME)
    dic_emoji_sent_away = useful_methods.EmojiSentimentCount(dfGame, 'away', START_TIME, END_TIME)

    return (dic_emoji_sent_home, dic_emoji_sent_away)

In [11]:
CountGameEmolexEmoji(week, home_team, away_team)

({'neg': 65, 'neut': 15, 'pos': 290}, {'neg': 198, 'neut': 500, 'pos': 2868})

In [12]:

# coding: utf-8
import os
import sys
import time
import datetime
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing


# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import emolex
import useful_methods


# *******************************************************
# *******************************************************

# Limitations
TIME_LIMIT = 63
RETWEET_STATUS = False
FILTER_STATUS = True
START_TIME = 1
END_TIME = 63


# *******************************************************
# *******************************************************

# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')
dfGameInfos = useful_methods.DropNanGames(dfGameInfos)


# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]


# Read Emotion-Lexicon-Soccer as Dictionary
dic_emolex_soccer = emolex.EmolexSoccerDic()


# *******************************************************
# *******************************************************

# ### Emolex Count Functions


# Summing counted emolex
def EmolexSumList(dfEmolex, start=1, end=60):
    # Time Interval
    dfEmolex.ith_minute = [int(ith_minute) for ith_minute in list(dfEmolex.ith_minute)]
    dfEmolex = dfEmolex[(dfEmolex.ith_minute >= start) & (dfEmolex.ith_minute <= end)]

    # Sum Emolex Count
    anger = dfEmolex.anger.sum()
    fear = dfEmolex.fear.sum()
    disgust = dfEmolex.disgust.sum()
    sadness = dfEmolex.sadness.sum()
    surprise = dfEmolex.surprise.sum()
    trust = dfEmolex.trust.sum()
    joy = dfEmolex.joy.sum()
    anticipation = dfEmolex.anticipation.sum()
    positive = dfEmolex.positive.sum()
    negative = dfEmolex.negative.sum()

    return np.array([anger, fear, disgust, sadness,
            surprise,
            trust, joy, anticipation,
            positive, negative])


# Count Home, Away Emolex
def CountGameEmolex(week, team_home, team_away):

    # Read Single as DF
    dfGame = useful_methods.SingleGameDf(week, team_home, team_away, filtering=FILTER_STATUS, retweet=RETWEET_STATUS)
    if dfGame is None:
        return (None, None)

    dfGame.ith_minute = [int(ith_minute) for ith_minute in list(dfGame.ith_minute)]

    # Count Emolex Words
    dfEmolexHome = emolex.CreateEmolexDF(dfGame[(dfGame.side == 'home') & (dfGame.ith_minute <= TIME_LIMIT)], dic_emolex_soccer)
    dfEmolexAway = emolex.CreateEmolexDF(dfGame[(dfGame.side == 'away') & (dfGame.ith_minute <= TIME_LIMIT)], dic_emolex_soccer)

    # Sum Emolex
    dic_emolex_home = EmolexSumList(dfEmolexHome, start=START_TIME, end=END_TIME)
    dic_emolex_away = EmolexSumList(dfEmolexAway, start=START_TIME, end=END_TIME)

    return (dic_emolex_home, dic_emolex_away)


# *******************************************************
# *******************************************************
def EmolexCountSingleMatch(ith_row):
    # Team names
    week = dfGameInfos.iloc[ith_row]['GW']
    team_home = dfGameInfos.iloc[ith_row]['home_team']
    team_away = dfGameInfos.iloc[ith_row]['away_team']

    # Count Emolex Word
    emolex_count = CountGameEmolex(week, team_home, team_away)

    # print each rows
    print('%s, %s, %s, %s, %s' %
        (week, team_home, team_away, emolex_count[0], emolex_count[1]))

    result = (week, team_home, team_away, emolex_count[0], emolex_count[1])

    return result

In [21]:
# Count Home, Away Emolex
def CountGameEmolexEmoji(week, team_home, team_away):

    # Read Single as DF
    dfGame = useful_methods.SingleGameDf(week, team_home, team_away, filtering=FILTER_STATUS, retweet=RETWEET_STATUS)
    if dfGame is None:
        return (None, None)

    dfGame.ith_minute = [int(ith_minute) for ith_minute in list(dfGame.ith_minute)]

    # Count Emolex Words
    dfEmolexHome = emolex.CreateEmolexDF(dfGame[(dfGame.side == 'home') & (dfGame.ith_minute <= TIME_LIMIT)], dic_emolex_soccer)
    dfEmolexAway = emolex.CreateEmolexDF(dfGame[(dfGame.side == 'away') & (dfGame.ith_minute <= TIME_LIMIT)], dic_emolex_soccer)

    # Sum Emolex
    dic_emolex_home = EmolexSumList(dfEmolexHome, start=START_TIME, end=END_TIME)
    dic_emolex_away = EmolexSumList(dfEmolexAway, start=START_TIME, end=END_TIME)

    # Count Emoji Sentiment Count
    dic_emoji_sent_home = useful_methods.EmojiSentimentCount(dfGame, 'home', START_TIME, END_TIME)
    dic_emoji_sent_away = useful_methods.EmojiSentimentCount(dfGame, 'away', START_TIME, END_TIME)

    # Merge: Emolex & Emoji Sentiment
    dic_emolex_home[-2] += dic_emoji_sent_home['pos']
    dic_emolex_home[-1] += dic_emoji_sent_home['neg']

    dic_emolex_away[-2] += dic_emoji_sent_away['pos']
    dic_emolex_away[-1] += dic_emoji_sent_away['neg']

    return (dic_emolex_home, dic_emolex_away)

In [25]:
useful_methods.EmojiSentimentCount(df, 'away', START_TIME, END_TIME)

{'neg': 198, 'neut': 500, 'pos': 2868}

In [15]:
dic1, dic2 = CountGameEmolex(week, 'City', 'Liverpool')

In [19]:
dic1

array([ 421.,  402.,  353.,  403.,  282.,  418.,  335.,  399.,  799.,  819.])

In [20]:
dic2

array([ 1040.,  1065.,   787.,  1008.,  1363.,  2659.,  2857.,  2334.,
        4939.,  2174.])

In [23]:
CountGameEmolexEmoji(week, 'City', 'Liverpool')

(array([  421.,   402.,   353.,   403.,   282.,   418.,   335.,   399.,
         1089.,   884.]),
 array([ 1040.,  1065.,   787.,  1008.,  1363.,  2659.,  2857.,  2334.,
         7807.,  2372.]))